# Create NCDPI 2013-2014 Raw Datasets
** This program downloads all original datasets from www.ncpublicschools.org and saves them as .csv files.  These data files are used to create all the flattened and machine learning datasets within the NCEA repository.**
1. This notebook downloads raw datasets directly from NCDPI specific URLs.
2. Each raw dataset is filtered by school year and saved in the original layout as a .csv file.
3. For consistency, both the Year and School code fields are renamed to "year" and "unit_code" in all files. 
4. All masking is removed from raw data fields using the following code: replace({"*":0, ">95":100, "<5":0, "<10":5 }) 
5. All * or carriage returns are removed from column names.  
6. Duplicate column names in accDrillDown files are renamed to include _Ct at the end for all count fields.
7. All raw datasets created by this program are used to create the "flattened" and "machine learning" Public School datasets.  

In [1]:
#import required Libraries
import pandas as pd
import numpy as np
import os
import string

#**********************************************************************************
# Set the following variables before running this code!!!
#**********************************************************************************

#Location where copies of the raw data files will be downloaded and saved as csv files.
dataDir = 'C:/Users/Jake/Documents/GitHub/EducationDataNC/2014/Raw Datasets/'

#All raw data files are filtered for the year below
schoolYear = 2014

## Process the Raw Data
**This section downloads raw data files directly from ncpublicschools.org or reads the raw data in from disk .**  
* The file output location is specified at the *dataDir* parameter.
* The *schoolYear* parameter is used to filter out the correct school year from raw data files that contain multiple years.

### Data Sources
* http://www.ncpublicschools.org/accountability/reporting/
* http://www.ncpublicschools.org/src/researchers/

In [2]:
#Download the raw data OR read in the .csv files at the dataDir path

#Profile Table filtered by schoolYear (years 2006-2017 at this URL)
url="http://www.ncpublicschools.org/docs/src/researchers/profile.xlsx"
profile = pd.read_excel(url, dtype={'unit_code': object})
profile = profile[profile.year == schoolYear]
profile.to_csv(dataDir + 'profile.csv', sep=',', index=False) 

#Profile Metric Table filtered by schoolYear (years 2002-2017 at this URL)  
url="http://www.ncpublicschools.org/docs/src/researchers/profile-metrics.xlsx"
profileMetric = pd.read_excel(url, dtype={'unit_code': object})
profileMetric = profileMetric[profileMetric.year == schoolYear]
profileMetric.to_csv(dataDir + 'profile-metrics.csv', sep=',', index=False) 

#Funding Table (years 2002-2017 at this URL) 
url='http://www.ncpublicschools.org/docs/src/researchers/funding.xlsx'
funding = pd.read_excel(url, dtype={'unit_code': object})
funding = funding[funding.year == schoolYear]
funding.to_csv(dataDir + 'funding.csv', sep=',', index=False) 

#School Performance Grade (SPG) (years 2014-2017)
if schoolYear == 2017: url = 'http://www.ncpublicschools.org/docs/accountability/reporting/2017/reports/spg1617.xlsx'
elif schoolYear == 2016: url = 'http://www.ncpublicschools.org/docs/accountability/reporting/spg1516.xlsx'
elif schoolYear == 2015: url = 'http://www.ncpublicschools.org/docs/accountability/reporting/spg1415.xlsx'
elif schoolYear == 2014: url = 'http://www.ncpublicschools.org/docs/accountability/reporting/spg1314.xlsx'

#This file has header data above the field names
if schoolYear == 2014: startLoc=4
elif schoolYear == 2015: startLoc=7
else: startLoc = 6

spg = pd.read_excel(url, dtype={'School Code' : object}, skiprows=startLoc)
#Remove *, \r, and \n from column names 
spg.rename(columns=lambda x: x.replace('*','').strip(), inplace=True)  
spg.rename(columns=lambda x: x.replace('\r',' '), inplace=True) 
spg.rename(columns=lambda x: x.replace('\n',' '), inplace=True) 
#Remove all masking from numeric fields
spg = spg.replace({"*":0, ">95":100, "<5":0, "<10":5 })
#Rename primary key to unit_code
spg = spg.rename({'School Code':'unit_code'}, axis=1)
#Save raw data to csv
spg.to_csv(dataDir + 'spg.csv', sep=',', index=False)  
#Read the file back in from disk to get the correct data types now that the masking is removed. 
spg = pd.read_csv(dataDir + 'spg.csv', low_memory=False, dtype={'unit_code': object})    

#READY Accountability Drill Down (each year 2014-2017 is in a different file)
if schoolYear == 2017: url = 'http://www.ncpublicschools.org/docs/accountability/reporting/2017/reports/accdrilldwn17.xlsx'
elif schoolYear == 2016: url = 'http://www.ncpublicschools.org/docs/accountability/reporting/accdrilldwn16.xlsx'
elif schoolYear == 2015: url = 'http://www.ncpublicschools.org/docs/accountability/reporting/accdrilldwn15.xlsx'
elif schoolYear == 2014: url = 'http://www.ncpublicschools.org/docs/accountability/reporting/acctdrilldwn14.xlsx'
#This file has header data above the field names
if schoolYear == 2014: startLoc=4 
else: startLoc = 6

accDrillDown = pd.read_excel(url, dtype={'School Code': object}, skiprows=startLoc)
#Rename duplicate columns names to be more descriptive 
accDrillDown.rename(columns=lambda x: x.replace('.1','_Ct').strip(), inplace=True)
#Rename primary key to unit_code
accDrillDown = accDrillDown.rename({'School Code':'unit_code'}, axis=1)
#Remove all masking from numeric fields
accDrillDown = accDrillDown.replace({"*":0, ">95":100, "<5":0, "<10":5 })
#Save the new file as .csv
accDrillDown.to_csv(dataDir + 'accDrillDown.csv', sep=',', index=False)
#Read the file back in from disk to get the correct data types now that the masking is removed. 
accDrillDown = pd.read_csv(dataDir + 'accDrillDown.csv', low_memory=False, dtype={'unit_code': object}) 

#Read To Achieve (RTA) (years 2014-2017 at this URL)
url='http://www.ncpublicschools.org/docs/src/researchers/rta.xlsx'
rta = pd.read_excel(url, dtype={'unit_code': object})
rta = rta[rta.year == schoolYear]
rta.to_csv(dataDir + 'rta.csv', sep=',', index=False)

#Participation Targets Overall Table (years 2013-2017 at this URL)
url='http://www.ncpublicschools.org/docs/accountability/reporting/participation-targets.xlsx'
pTargets = pd.read_excel(url, dtype={'unit_code': object})
pTargets = pTargets[pTargets.year == schoolYear]
pTargets.to_csv(dataDir + 'participation-targets.csv', sep=',', index=False)

#School Indicators Table (years 2002-2017 at this URL)
url='http://www.ncpublicschools.org/docs/src/researchers/school-indicators.xlsx'
schoolInds = pd.read_excel(url, dtype={'Unit_Code': object})
#Rename primary key and year for consistency
schoolInds = schoolInds.rename({'Unit_Code':'unit_code','Year':'year'}, axis=1)
schoolInds = schoolInds[schoolInds.year == schoolYear]
schoolInds.to_csv(dataDir + 'school-indicators.csv', sep=',', index=False)

#Specialized Course Enrollment Table (years 2014-2017 at this URL) 
url='http://www.ncpublicschools.org/docs/src/researchers/sce.xlsx'
sce = pd.read_excel(url, dtype={'unit_code': object})
sce = sce[sce.year == schoolYear]
sce.to_csv(dataDir + 'sce.csv', sep=',', index=False)

#College Enrollment Table (years 2011-2015 at this URL)  
#Tracks college enrollment within 16 mos of graduation
url='http://www.ncpublicschools.org/docs/src/researchers/college-enrollment.xlsx'
collegeEnroll = pd.read_excel(url, dtype={'unit_code': object})
#Most recent graduation year runs two years behind current school year in this table 
collegeEnroll = collegeEnroll[collegeEnroll.graduation_year == schoolYear - 2]
collegeEnroll.to_csv(dataDir + 'college-enrollment.csv', sep=',', index=False)

#Environment Table (years 2006-2017 at this URL)
url='http://www.ncpublicschools.org/docs/src/researchers/environment.xlsx'
environment = pd.read_excel(url, dtype={'unit_code': object})
environment= environment[environment.year == schoolYear]
environment.to_csv(dataDir + 'environment.csv', sep=',', index=False)

#Personnel Table (years 2006-2017 at this URL)
url='http://www.ncpublicschools.org/docs/src/researchers/personnel.xlsx'
personnel = pd.read_excel(url, dtype={'unit_code': object})
personnel= personnel[personnel.year == schoolYear]
personnel.to_csv(dataDir + 'personnel.csv', sep=',', index=False)

#Educator Experience Table (YOE) (years 2006-2017 at this URL)
url='http://www.ncpublicschools.org/docs/src/researchers/yoe.xlsx'
yoe = pd.read_excel(url, dtype={'unit_code': object})
yoe = yoe[yoe.year == schoolYear]
yoe.to_csv(dataDir + 'yoe.csv', sep=',', index=False)

#Educator Effectiveness (years 2011-2017 at this URL)
url='http://www.ncpublicschools.org/docs/src/researchers/effectiveness.xlsx'
effectiveness = pd.read_excel(url, dtype={'unit_code': object})
effectiveness = effectiveness[effectiveness.year == schoolYear]
effectiveness.to_csv(dataDir + 'effectiveness.csv', sep=',', index=False)

#Statistical Profiles - Student Body Racial Compositions at the School Level
import io
import requests
url='http://apps.schools.nc.gov/ords/f?p=145:221::CSV::::'
#Passing this URL directly into pd.read_csv() threw HTTP errors - This is my workaround
s = requests.get(url).content
ec_pupils = pd.read_csv(io.StringIO(s.decode('utf-8')), low_memory=False
                        , dtype={'LEA': object,'School': object})
#Rename year for consistency
ec_pupils.rename({'Year':'year'}, axis=1, inplace=True)
ec_pupils = ec_pupils[ec_pupils.year == schoolYear]
#Create unit_code from LEA and School code as an index
ec_pupils['unit_code'] = ec_pupils['LEA'] + ec_pupils['School']
#Save the original data to disk 
ec_pupils.to_csv(dataDir + 'ec_pupils.csv', sep=',', index=False)